In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import paper_classes_2 as pc
import Post_processing as pp
import itertools
import pickle


In [2]:
def plot_dispatch(pv, demand, E, week=30):
    """ Visualize dispatch algorithm for a specific week
    Parameters:
        demand (pd.Series): demand production
        pv (pd.Series): pv production
        E (dict):  Energy flows. Dictionary of pd.Series: res_pv, grid2load, store2inv, LevelOfCharge
    """

    sliced_index = (pv.index.week==week)
    pv_sliced = pv[sliced_index]
    demand_sliced = demand[sliced_index]
    self_consumption = E['inv2load'][sliced_index]
    
    direct_self_consumption = np.minimum(pv_sliced,demand_sliced)# E['inv2load'][sliced_index]
    indirect_self_consumption = self_consumption-direct_self_consumption
    res_pv_sliced = E['res_pv'][sliced_index]
    grid2load_sliced = E['grid2load'][sliced_index]
    store2inv_sliced = E['store2inv'][sliced_index]
    LevelOfCharge = E['LevelOfCharge'][sliced_index]
    inv2grid = E['inv2grid'][sliced_index]
    grid2load = E['grid2load'][sliced_index]
    aux=np.maximum(0,self_consumption)

    fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(17, 4*3), frameon=False,
                             gridspec_kw={'height_ratios': [3, 1, 1], 'hspace': 0.04})

    #fig, ax = plt.subplots(figsize=(17, 4))
    axes[0].plot(demand_sliced.index, demand_sliced, color='black', lw=2,label='demand')
    axes[0].plot(pv_sliced.index, pv_sliced, color='black',ls='--', lw=2,label='PV')
    axes[0].fill_between(direct_self_consumption.index, 0, direct_self_consumption, color='orange', alpha=.8, label='DSC')
    axes[0].fill_between(pv_sliced.index, self_consumption, pv_sliced , where=pv_sliced<demand_sliced,color='blue', hatch='//',
                         alpha=.3,label='ISC')
    axes[0].fill_between(pv_sliced.index, direct_self_consumption, pv_sliced ,where=pv_sliced>demand_sliced, color='gold', alpha=.3,label='Excess PV')

    axes[0].fill_between(grid2load_sliced.index,self_consumption,demand_sliced,color='red',alpha=.2, label='grid2load')
        

    #axes[0].plot(grid2load_sliced.index, grid2load_sliced, color='red', ls=":", lw=1)
    axes[0].set_ylim([0, axes[0].get_ylim()[1] ])
    axes[0].set_ylabel('Power (kW)')

    axes[1].fill_between(LevelOfCharge.index, 0, LevelOfCharge, color='grey', alpha=.2, label='SOC')
    axes[1].set_ylabel('State of Charge (kWh)')

    axes[2].fill_between(inv2grid.index, 0, inv2grid, color='green', alpha=.2,label='injected2grid')
    axes[2].fill_between(inv2grid.index, 0, -grid2load, color='red', alpha=.2,label='grid drawn')
    axes[2].set_ylabel('In/out from grid (kW)')
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    return



In [3]:
def plot_dispatch(pv, demand, E,param, week=30):
    """ Visualize dispatch algorithm for a specific week
    Parameters:
        demand (pd.Series): demand production
        pv (pd.Series): pv production
        E (dict):  Energy flows. Dictionary of pd.Series: res_pv, grid2load, store2inv, LevelOfCharge
    """

    sliced_index = (pv.index.week==week)
    pv_sliced = pv[sliced_index]
    demand_sliced = demand[sliced_index]
    self_consumption = E['inv2load'][sliced_index]
    if 'store2grid' in E.keys():
        store2grid_sliced=E['store2grid'][sliced_index]*param['InverterEfficiency']
    direct_self_consumption = np.minimum(pv_sliced,demand_sliced)# E['inv2load'][sliced_index]
    indirect_self_consumption = self_consumption-direct_self_consumption
    res_pv_sliced = E['res_pv'][sliced_index]
    grid2load_sliced = E['grid2load'][sliced_index]
    store2inv_sliced = E['store2inv'][sliced_index]
    LevelOfCharge = E['LevelOfCharge'][sliced_index]
    inv2grid = E['inv2grid'][sliced_index]
    grid2load = E['grid2load'][sliced_index]
    aux=np.maximum(0,self_consumption)

    fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(17, 4*3), frameon=False,
                             gridspec_kw={'height_ratios': [3, 1, 1], 'hspace': 0.04})

    #fig, ax = plt.subplots(figsize=(17, 4))
    axes[0].plot(demand_sliced.index, demand_sliced, color='black', lw=2,label='demand')
    axes[0].plot(pv_sliced.index, pv_sliced, color='black',ls='--', lw=2,label='PV')
    axes[0].fill_between(direct_self_consumption.index, 0, direct_self_consumption, color='orange', alpha=.8, label='DSC')
    axes[0].fill_between(pv_sliced.index, self_consumption, pv_sliced , where=pv_sliced<demand_sliced,color='blue', hatch='//',
                         alpha=.3,label='ISC')
    axes[0].fill_between(pv_sliced.index, direct_self_consumption, pv_sliced ,where=pv_sliced>demand_sliced, color='gold', alpha=.3,label='Excess PV')

    axes[0].fill_between(grid2load_sliced.index,self_consumption,demand_sliced,color='red',alpha=.2, label='grid2load')

    #axes[0].plot(grid2load_sliced.index, grid2load_sliced, color='red', ls=":", lw=1)
    axes[0].set_ylim([0, axes[0].get_ylim()[1] ])
    axes[0].set_ylabel('Power (kW)')

    axes[1].fill_between(LevelOfCharge.index, 0, LevelOfCharge, color='grey', alpha=.2, label='SOC')
    axes[1].set_ylabel('State of Charge (kWh)')
    if 'store2grid' in E.keys():
        axes[2].fill_between(store2grid_sliced.index, 0, store2grid_sliced, color='blue', alpha=.2,label='store2grid')

        axes[2].fill_between(inv2grid.index, store2grid_sliced, inv2grid, color='green', alpha=0.2,label='PV2grid')
    else:
        axes[2].fill_between(inv2grid.index, 0, inv2grid, color='green', alpha=0.2,label='PV2grid')
    axes[2].fill_between(inv2grid.index, 0, -grid2load, color='red', alpha=.2,label='grid drawn')
    axes[2].set_ylabel('In/out from grid (kW)')
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    return



In [4]:
def print_analysis_orig(pv, demand, param, E):
    timestep = param['timestep']
    SelfConsumption = np.sum(E['inv2load']) * timestep
    TotalFromGrid = np.sum(E['grid2load']) * timestep
    TotalToGrid = np.sum(E['inv2grid']) * timestep
    TotalLoad = demand.sum() * timestep
    TotalPV = pv.sum() * timestep
    TotalBatteryGeneration = np.sum(E['store2inv']) * timestep
    TotalBatteryConsumption = np.sum(E['pv2store']) * timestep
    BatteryLosses=E['batt_losses'].sum()
    InverterLosses=E['inv_losses'].sum()
    SelfConsumptionRate = SelfConsumption / TotalPV * 100             # in %
    SelfSufficiencyRate = SelfConsumption / TotalLoad * 100
    AverageDepth = TotalBatteryGeneration / (365 * param['BatteryCapacity'])
    Nfullcycles = 365 * AverageDepth
    residue = TotalPV + TotalFromGrid - TotalToGrid - BatteryLosses - InverterLosses - TotalLoad-E['LevelOfCharge'][-1]*( param['InverterEfficiency'])

    print ('Total yearly consumption: {:1g} kWh'.format(TotalLoad))
    print ('Total PV production: {:1g} kWh'.format(TotalPV))
    print ('Self Consumption: {:1g} kWh'.format(SelfConsumption))
    print ('Total fed to the grid: {:1g} kWh'.format(TotalToGrid))
    print ('Total bought from the grid: {:1g} kWh'.format(TotalFromGrid))
    print ('Self consumption rate (SCR): {:1g}%'.format(SelfConsumptionRate))
    print ('Self sufficiency rate (SSR): {:1g}%'.format(SelfSufficiencyRate))
    print ('Amount of energy provided by the battery: {:1g} kWh'.format(TotalBatteryGeneration))
    print ('Average Charging/Discharging depth: {:1g}'.format(AverageDepth))
    print ('Number of equivalent full cycles per year: {:1g} '.format(Nfullcycles))
    print ('Total battery losses: {:1g} kWh'.format(BatteryLosses))
    print ('Total inverter losses: {:1g} kWh'.format(InverterLosses))
    print ('Residue (check): {:1g} kWh'.format(residue))

In [5]:

def print_analysis(pv, demand, param, E):
    """ Print statistics and information of the dispatched solution
    Arguments
        pv (pd.Series): PV timeseries
        demand (pd.Series): demand timeseries
        param (dict): dictionary of technical parameters
        E (dict): dictionary of energy flows as estimated by the algorithm
    Returns
        none
    """
    timestep = param['timestep']
    
    TotalFromGrid = np.sum(E['grid2load']) * timestep
    if 'store2grid' in E.keys():
        
        IndirectSelfConsumption=np.sum(E['store2load']) *param['InverterEfficiency']* timestep
        DirectSelfConsumption= np.sum(E['inv2load']) * timestep-IndirectSelfConsumption
        TotalToGridFromPV = (np.sum(E['inv2grid'])-np.sum(E['store2grid'])*param['InverterEfficiency']) * timestep
        TotalToGridFromBatt = (np.sum(E['inv2grid']) * timestep-TotalToGridFromPV)
    SelfConsumption = np.sum(E['inv2load']) * timestep
    TotalToGrid=(np.sum(E['inv2grid']))*timestep
    TotalLoad = demand.sum() * timestep
    TotalPV = pv.sum() * timestep
    TotalBatteryGeneration = np.sum(E['store2inv']) * timestep
    TotalBatteryConsumption = np.sum(E['pv2store']) * timestep
    BatteryLosses=E['batt_losses'].sum()
    InverterLosses=E['inv_losses'].sum()
    SelfConsumptionRate = SelfConsumption / TotalPV * 100             # in %
    SelfSufficiencyRate = SelfConsumption / TotalLoad * 100
    AverageDepth = TotalBatteryGeneration / (365 * param['BatteryCapacity'])
    Nfullcycles = 365 * AverageDepth
    if 'store2grid' in E.keys():
        store2grid=np.sum(E['store2grid'])*timestep

    residue = TotalPV + TotalFromGrid - TotalToGrid - BatteryLosses - InverterLosses - TotalLoad-E['LevelOfCharge'][-1]*( param['InverterEfficiency'])


    print ('Total yearly consumption: {:1g} kWh'.format(TotalLoad))
    print ('Total PV production: {:1g} kWh'.format(TotalPV))
    print ('Self Consumption: {:1g} kWh'.format(SelfConsumption))
    print ('Total fed to the grid: {:1g} kWh'.format(TotalToGrid))
    if 'store2grid' in E.keys():
        print ('Direct Self Consumption: {:1g} kWh'.format(DirectSelfConsumption))
        print ('Indirect Self Consumption: {:1g} kWh'.format(IndirectSelfConsumption))
        print ('Total fed to the grid from PV: {:1g} kWh'.format(TotalToGridFromPV))
        print ('Total fed to the grid from battery: {:1g} kWh'.format(TotalToGridFromBatt))

    print ('Total bought from the grid: {:1g} kWh'.format(TotalFromGrid))
    print ('Self consumption rate (SCR): {:.3g}%'.format(SelfConsumptionRate))
    print ('Self sufficiency rate (SSR): {:.3g}%'.format(SelfSufficiencyRate))
    print ('Amount of energy provided by the battery: {:1g} kWh'.format(TotalBatteryGeneration))
    print ('Average Charging/Discharging depth: {:1g}'.format(AverageDepth))
    print ('Number of equivalent full cycles per year: {:1g} '.format(Nfullcycles))
    print ('Total battery losses: {:1g} kWh'.format(BatteryLosses))
    print ('Total inverter losses: {:1g} kWh'.format(InverterLosses))

    print ('Residue (check): {:1g} kWh'.format(residue))


In [6]:

def dispatch_max_sc(pv, demand, param, return_series=False):
    """ Self consumption maximization pv + battery dispatch algorithm.
    The dispatch of the storage capacity is performed in such a way to maximize self-consumption:
    the battery is charged when the PV power is higher than the load and as long as it is not fully charged.
    It is discharged as soon as the PV power is lower than the load and as long as it is not fully discharged.
    Arguments:
        pv (pd.Series): Vector of PV generation, in kW DC (i.e. before the inverter)
        demand (pd.Series): Vector of household consumption, kW
        param (dict): Dictionary with the simulation parameters:
                timestep (float): Simulation time step (in hours)
                BatteryCapacity: Available battery capacity (i.e. only the the available DOD), kWh
                BatteryEfficiency: Battery round-trip efficiency, -
                InverterEfficiency: Inverter efficiency, -
                MaxPower: Maximum battery charging or discharging powers (assumed to be equal), kW
        return_series(bool): if True then the return will be a dictionary of series. Otherwise it will be a dictionary of ndarrays.
                        It is reccommended to return ndarrays if speed is an issue (e.g. for batch runs).
    Returns:
        dict: Dictionary of Time series
    """

    bat_size_e_adj = param['BatteryCapacity']
    bat_size_p_adj = param['MaxPower']
    n_bat = param['BatteryEfficiency']
    n_inv = param['InverterEfficiency']
    timestep = param['timestep']
    # We work with np.ndarrays as they are much faster than pd.Series
    Nsteps = len(pv)
    LevelOfCharge = np.zeros(Nsteps)
    pv2store = np.zeros(Nsteps)
    inv2grid = np.zeros(Nsteps)
    store2inv = np.zeros(Nsteps)
    #store2load = np.zeros(Nsteps)
    grid2store = np.zeros(Nsteps) # TODO Always zero for now.
    #store2grid = np.zeros(Nsteps)
    batt_losses = np.zeros(Nsteps)
    #prices=np.ones(Nsteps)*.07
    #Load served by PV
    pv2inv = np.minimum(pv, demand / n_inv)  # DC direct self-consumption

    #Residual load
    res_load = (demand - pv2inv * n_inv)  # AC
    inv2load = pv2inv * n_inv  # AC

    #Excess PV
    res_pv = np.maximum(pv - demand/n_inv, 0)  # DC
    #PV to storage after eff losses
    pv2inv = pv2inv.values

    #first timestep = 0
    LevelOfCharge[0] = 0  # bat_size_e_adj / 2  # DC
    
    for i in range(1,Nsteps):
        #PV to storage
        if LevelOfCharge[i-1] >= bat_size_e_adj:  # if battery is full
                pv2store[i] = 0
        else: #if battery is not full
            if LevelOfCharge[i-1] + res_pv[i] * timestep > bat_size_e_adj:  # if battery will be full after putting excess
                pv2store[i] = min((bat_size_e_adj - LevelOfCharge[i-1]) / timestep, bat_size_p_adj)
            else:
                pv2store[i] = min(res_pv[i], bat_size_p_adj)

        #Storage to load or grid
        ######################################
        batt_losses[i]=pv2store[i]*(1-n_bat)
        if pv2store[i]==0:#not charging
            store2inv[i] = min(bat_size_p_adj,  # DC
                           res_load[i] / n_inv,
                           LevelOfCharge[i-1] / timestep)
            
            
        ######################################a
        
        #SOC
        LevelOfCharge[i] = min(LevelOfCharge[i-1] - batt_losses[i]-(store2inv[i] - pv2store[i] - grid2store[i]) * timestep,  # DC
                               bat_size_e_adj)
    
    pv2inv = pv2inv + res_pv - pv2store
    inv2load = inv2load + store2inv * n_inv  # AC
    inv2grid = (res_pv - pv2store) * n_inv  # AC
    grid2load = demand - inv2load  # AC
    inv_losses=(pv2inv+store2inv)*(1-n_inv)
    #MaxDischarge = np.minimum(LevelOfCharge[i-1]*BatteryEfficiency/timestep,MaxPower)


    #Potential Grid to storage  # TODO: not an option for now in this strategy
    # GridPurchase = False

    out = {'pv2inv': pv2inv,
            'res_pv': res_pv,
            'pv2store': pv2store,
            'inv2load': inv2load,
            'grid2load': grid2load,
            'store2inv': store2inv,
            'LevelOfCharge': LevelOfCharge,
            'inv2grid': inv2grid,
            'batt_losses':batt_losses,
            'inv_losses':inv_losses
            # 'grid2store': grid2store
            }
    if not return_series:
        out_pd = {}
        for k, v in out.items():  # Create dictionary of pandas series with same index as the input pv
            out_pd[k] = pd.Series(v, index=pv.index)
        out = out_pd
    return out

In [7]:
def flag_selection(df,list_product,community_size,list_pv_penetration,list_batt_penetration):
    list_names=['sub_'+str(i[0])+'_'+str(i[1]) for i in list_product]
    for i in list_names:
        df[i]=False
    dict_pv_df={}
    dict_batt_df={}
    j=0
    l=0
    for i in list_pv_penetration: 
        #print('-----')
        dict_pv_df[j]=np.random.choice(df.index,int(len(df)*(i/100)), replace=False)
        for k in list_batt_penetration:
            #print(int(len(dict_pv_df[j])*(k/100)))
            dict_batt_df[l]=np.random.choice(dict_pv_df[j],int(len(dict_pv_df[j])*(k/100)), replace=False)
            df.iloc[dict_batt_df[l],l+2]=True
            l+=1
        j+=1
    return df

In [8]:
def save_obj(obj, name ):
    with open('Switzerland/Output/'+name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('Switzerland/Output/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [9]:
def run_comm_as_today(first_time):
    community_size=100
    timestep=1 # in hours
    pv_penetration=100
    batt_penetration=100
    list_pv_penetration=[100,75,50,25]# %
    list_batt_penetration=[100,75,50,25]# %
    df_CH=pd.read_csv('../BASOPRA_IRES/Input/df_CH_Marzia2.csv', encoding='utf8', sep=',',usecols=[*range(0, 669)],
                                      engine='python',date_parser=lambda col: pd.to_datetime(col, utc=True),infer_datetime_format=True,index_col=0)
    df_CH.index=df_CH.index.tz_convert('Europe/Brussels')
    if timestep==1:
        df_CH=df_CH.resample('1H').sum()
    
    if first_time:
        max_PV=10
        path='Switzerland/Input/'
        df_pv_ch=pd.read_excel(path+'PV_beneficiaires_Swiss.xlsx')
        df_nat=df_pv_ch[(df_pv_ch['Anlage_Projekt-Bezeichnung']=='natürliche Person')]
        df_nat_pv=df_nat[(df_nat['Anlage_Energieträger']=='Photovoltaik')]
        df_nat_pv_15=df_nat_pv[df_nat_pv['Leistung [kW]']<max_PV]['Leistung [kW]'].reset_index(drop=True)
        df_names=pd.DataFrame(df_CH.loc[:,(df_CH.sum()<7500)&(df_CH.columns!='E_PV')].sample(n=community_size,axis=1).columns)
        
        selection=pd.concat([df_nat_pv_15.round(1).sample(n=community_size).reset_index(drop=True),df_names],axis=1)
        selection.columns=['PV_size','name']
        list_product=list(itertools.product(list_pv_penetration,list_batt_penetration))
        flag_selection(selection,list_product,community_size,list_pv_penetration,list_batt_penetration)
        #print(selection.sum())
        selection.to_csv('Switzerland/Input/selection.csv',index=False)
    else:#Doesnt work TODO
        selection=pd.read_csv('Switzerland/Input/selection.csv')
        #pd.read_csv('../BASOPRA_IRES/Input/df_CH_Marzia2.csv', encoding='utf8', sep=',',usecols=demand_profiles['0'],engine='python')
    list_names=[str(i) for i in selection.iloc[:,1]]
    param_tech = {'BatteryCapacity': 10,
                  'BatteryEfficiency': .91,
                  'InverterEfficiency': .94,
                  'timestep': timestep,
                  'MaxPower': 4
                 }

    # Community as today (Current state in Community)
    # Select the PV and battery penetration. Here there is no battery export.
    #We use the model from Quoilin et al. [1] for PV and battery dispatch and adapt it for the community case.
    #[1] Quoilin, S. et al., 'Quantifying self-consumption linked to solar home battery systems: Statistical analysis and economic assessment', Applied Energy, Elsevier, 2016, 182, pp. 58-67 (https://github.com/energy-modelling-toolkit/prosumpy)
    param_tech_no_batt=param_tech.copy()
    param_tech_no_batt.update({'BatteryCapacity':0,'MaxPower':0})
    
    # We set the battery capacity to zero in some cases to get the direct SC only
    # We iterate through the community and get the dispatching

    for pv_penetration in list_pv_penetration:
        for batt_penetration in list_batt_penetration:
            print('########################################')
            print('PV penetration {}'.format(pv_penetration))

            print('Battery penetration {}'.format(batt_penetration))
            nested_out={}
            j=0
            k=0
            PV_size_comm=0
            for i in selection.index:
                #print(i, end='')
                if selection.loc[i,'sub_'+str(pv_penetration)+'_100']:#all with PV

                    PV_size_comm+=selection.PV_size[i]
                    if selection.loc[i,'sub_'+str(pv_penetration)+'_'+str(batt_penetration)]: #if battery
                        nested_out[i]=dispatch_max_sc(df_CH.E_PV*selection.PV_size[i],df_CH.loc[:,str(selection.name[i])],param_tech)
                        j+=1
                    else: #if only PV battery=0 kWh
                        nested_out[i]=dispatch_max_sc(df_CH.E_PV*selection.PV_size[i],df_CH.loc[:,str(selection.name[i])],param_tech_no_batt)
                        k+=1
                else: #No PV
                    nested_out[i]=dispatch_max_sc(df_CH.E_PV*0,df_CH.loc[:,str(selection.name[i])],param_tech_no_batt)

            #get all the data in a single dict
            pv2inv=pd.DataFrame()
            res_pv=pd.DataFrame()
            pv2store=pd.DataFrame()
            inv2load=pd.DataFrame()
            grid2load=pd.DataFrame()
            store2inv=pd.DataFrame()
            LevelOfCharge=pd.DataFrame()
            inv2grid=pd.DataFrame()
            batt_losses=pd.DataFrame()
            inv_losses=pd.DataFrame()
            for i in nested_out.keys():
                pv2inv['pv2inv'+str(i)]=nested_out[i]['pv2inv']
                pv2store['pv2store'+str(i)]=nested_out[i]['pv2store']
                inv2load['inv2load'+str(i)]=nested_out[i]['inv2load']
                store2inv['store2inv'+str(i)]=nested_out[i]['store2inv']
                LevelOfCharge['LevelOfCharge'+str(i)]=nested_out[i]['LevelOfCharge']
                batt_losses['batt_losses'+str(i)]=nested_out[i]['batt_losses']
                inv_losses['inv_losses'+str(i)]=nested_out[i]['inv_losses']
                res_pv['res_pv'+str(i)]=nested_out[i]['res_pv']#not important, recalculated later
                inv2grid['inv2grid'+str(i)]=nested_out[i]['inv2grid']#not important, recalculated later
                grid2load['grid2load'+str(i)]=nested_out[i]['grid2load']#not important, recalculated later

            nested_dict={'pv2inv':pv2inv.sum(axis=1),'res_pv':res_pv.sum(axis=1),'pv2store':pv2store.sum(axis=1),'inv2load':inv2load.sum(axis=1),
                         'grid2load':grid2load.sum(axis=1),'store2inv':store2inv.sum(axis=1),'LevelOfCharge':LevelOfCharge.sum(axis=1),
                         'inv2grid':inv2grid.sum(axis=1),'batt_losses':batt_losses.sum(axis=1),'inv_losses':inv_losses.sum(axis=1)}
            param_tech_no_batt.update({'InverterEfficiency':1})#nested_dict['inv2grid'] is PV generation already in AC used as input for next step
            out_comm_res=dispatch_max_sc(nested_dict['inv2grid'],nested_dict['grid2load'],param_tech_no_batt)
            out_comm_final={}
            out_comm_final['pv2inv']=nested_dict['pv2inv']
            out_comm_final['pv2store']=nested_dict['pv2store']
            out_comm_final['inv2load']=(nested_dict['inv2load']+out_comm_res['inv2load'])
            out_comm_final['store2inv']=nested_dict['store2inv']
            out_comm_final['LevelOfCharge']=nested_dict['LevelOfCharge']
            out_comm_final['res_pv']=out_comm_res['res_pv']
            out_comm_final['inv2grid']=out_comm_res['inv2grid']
            out_comm_final['grid2load']=out_comm_res['grid2load']
            out_comm_final['batt_losses']=nested_dict['batt_losses']
            out_comm_final['inv_losses']=nested_dict['inv_losses']

            out_comm_final['PV_size_comm']=PV_size_comm
            out_comm_final['df']=df_CH
            out_comm_final['param_tech']=param_tech
            out_comm_final['param_tech_no_batt']=param_tech_no_batt
            out_comm_final['nested_dict']=nested_dict
            out_comm_final['pv_penetration']=pv_penetration
            out_comm_final['batt_penetration']=batt_penetration
            out_comm_final['selection']=selection


            print_analysis(df_CH.E_PV*PV_size_comm, df_CH.loc[:,list_names].sum(axis=1),param_tech_no_batt,out_comm_final)
            save_obj(out_comm_final,'community_as_today'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
    return selection

In [11]:
#%%time
#selection_false=run_comm_as_today(False)

########################################
PV penetration 100
Battery penetration 100


/opt/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in double_scalars


Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 224438 kWh
Total fed to the grid: 406763 kWh
Total bought from the grid: 37671.2 kWh
Self consumption rate (SCR): 32.8%
Self sufficiency rate (SSR): 85.6%
Amount of energy provided by the battery: 126832 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: inf 
Total battery losses: 12574.2 kWh
Total inverter losses: 40289.4 kWh
Residue (check): -1.45462e-10 kWh
########################################
PV penetration 100
Battery penetration 75
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 199221 kWh
Total fed to the grid: 445052 kWh
Total bought from the grid: 62887.6 kWh
Self consumption rate (SCR): 29.1%
Self sufficiency rate (SSR): 76%
Amount of energy provided by the battery: 96167.2 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: inf 
Total battery losses: 9534.57 kWh
Total inverter 